In [ ]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

from data import load_data
from utils import confusion_matrix


In [ ]:
epochs = 30
batch_size = 16
n_hidden = 32

In [ ]:
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

X_train, X_test, Y_train, Y_test = load_data()
# train-test data split
# X는 input, Y는 (정답)output

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])

# 분류하고 싶은 클래스 개수
n_classes = _count_classes(Y_train) 

model = Sequential()

# n_hidden : 은닉계층 임베딩 되는 차원
# LSTM cell =  n_hidden 차원으로 만드는 것
# timestep : split해서 넣을 데이터 절편 크기
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# input차원 - (timesteps, input_dim)

model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

# loss : predicted 결과와 real 결과의 차이
# optimizer : local optimum에 빠지지 않도록 넘어가게 해주는 장치
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

# Evaluate
print(confusion_matrix(Y_test, model.predict(X_test)))